# Post-Classical Clean Up 4: Hyperion

In [1]:
label = 'hyperion'

In [2]:
from datetime import datetime

import pandas
import sqlalchemy

SITE_URL = 'https://musicbrainz.org'

# import postgresql environ variables
# defined in postgres-dockerfile/postgres.env
PGHOST = '127.0.0.1'
PGDATABASE = 'musicbrainz'
PGUSER = 'musicbrainz'
PGPASSWORD = 'musicbrainz'
PGPORT = 5432

def sql(query, **kwargs):
    """helper function for SQL queries using the %(...) syntax
    Parameters for the query must be passed as keyword arguments
    e.g. sql('SELECT * FROM artist WHERE name=%(singer)s', singer='Bob Dylan')
    """
    engine = sqlalchemy.create_engine(
        'postgresql+psycopg2://'
        '{PGUSER}:{PGPASSWORD}@{PGHOST}:{PGPORT}/{PGDATABASE}'.format(**globals()),
        isolation_level='READ UNCOMMITTED')
    return pandas.read_sql(query, engine, params=kwargs)

# helper function to build canonical URLs
def _mb_link(entity_type, mbid):
    return '<a href="{url}/{entity_type}/{mbid}">{mbid}</a>'.format(
        url=SITE_URL, **locals())

mb_release_link = lambda mbid: _mb_link('release', mbid) # noqa

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [4]:
df = sql("""SELECT id, gid, name, comment FROM label WHERE name IN ('hyperion', 'helios');""")
label_mbid = df.gid.tolist()
df.head()

,id,gid,name,comment
0,388,08e6c3c8-81ab-405f-9cff-10f6b8db064c,hyperion,UK classical
1,22708,0a94e96a-9219-4dd7-a529-18d34e77f50f,helios,UK classical


## New releases added after Feb. 1st

Actually I'm checking when "release label" links were added to release, which is most of the time done together.

In [5]:
new_releases = sql("""
SELECT editor.name AS editor,
       COUNT(*) AS edit_count
  FROM label
  JOIN edit_label  ON label.id = edit_label.label
  JOIN edit        ON edit.id = edit_label.edit
  JOIN editor      ON editor.id = edit.editor
 WHERE label.name IN ('hyperion', 'helios')
   AND edit.open_time >= '2018-02-01'
GROUP BY editor.name
ORDER BY edit_count DESC
   ;""")
new_releases

,editor,edit_count
0,loujin,74
1,ListMyCDs.com,38
2,reosarevok,35
3,mmirG,19
4,MetaTunes,14
5,steinbdj,9
6,CatCat,6
7,Griomo,5
8,stupidname,3
9,mhendu,3


In [6]:
new_releases.edit_count.sum()

213

## Recording edits

Find all edits between Feb. 1st and March 14th regarding recordings present on Hyperion/Helios releases

In [7]:
recording_edits = sql("""
SELECT editor.name AS editor,
       COUNT(*) AS edit_count
  FROM recording     AS rec
  JOIN edit_recording      ON rec.id = edit_recording.recording
  JOIN edit                ON edit.id = edit_recording.edit
  JOIN editor              ON editor.id = edit.editor
  JOIN track               on track.recording = rec.id
  JOIN medium        AS m  ON track.medium = m.id
  JOIN release       AS r  ON m.release = r.id
  JOIN release_label AS rl ON rl.release = r.id
  JOIN label               ON rl.label = label.id
 WHERE label.name IN ('hyperion', 'helios')
   AND edit.open_time >= '2018-02-01'
GROUP BY editor.name
ORDER BY edit_count DESC
   ;""")
recording_edits

,editor,edit_count
0,loujin,20189
1,ListMyCDs.com,11245
2,reosarevok,10609
3,stupidname,6652
4,CatCat,4097
5,dosoe,3513
6,obtext,2943
7,steinbdj,1330
8,MetaTunes,1024
9,Griomo,807


In [8]:
recording_edits.edit_count.sum()

65784

## Release covers added

In [9]:
sql("""
SELECT COUNT(*) AS edit_count
  FROM release        AS r
  JOIN edit_release        ON r.id = edit_release.release
  JOIN edit                ON edit.id = edit_release.edit
  JOIN release_label AS rl ON rl.release = r.id
  JOIN label               ON rl.label = label.id
 WHERE label.name IN ('hyperion', 'helios')
   AND edit.open_time >= '2018-02-01'
   AND edit.type = 314
   ;""")

,edit_count
0,377


## phonographic relationships added on recordings

NB: Phonographic copyright ℗ should use 'Hyperion Records Ltd', not Hyperion or Helios

In [10]:
sql("""
SELECT COUNT(*) AS edit_count
  FROM recording     AS r
  JOIN edit_recording      ON r.id = edit_recording.recording
  JOIN edit                ON edit.id = edit_recording.edit
  JOIN l_label_recording AS lar ON r.id = lar.entity1
  JOIN label                    ON label.id = lar.entity0
  JOIN link                     ON link.id = lar.link
 WHERE label.name = 'Hyperion Records Ltd, London'
   AND edit.open_time >= '2018-02-01'
   AND link.link_type = 867
;""")

,edit_count
0,25148


## Booklet editor relation

Relation created for this community sprint if I'm not mistaken

In [28]:
sql("""
SELECT a.name, COUNT(*) AS cnt
  FROM release            AS r
  JOIN l_artist_release   AS lar ON r.id = lar.entity1
  JOIN artist             AS a   ON a.id = lar.entity0
  JOIN link               AS l   ON l.id = lar.link
 WHERE l.link_type=929
 GROUP BY a.name
 ORDER BY cnt DESC
 LIMIT 5
""")

,name,cnt
0,Tim Parry,91
1,Nick Flower,8
2,Eleanor Wilson,5
3,Roswita Cervone,3
4,Peter Hall,3


4 of those 5 work for Hyperion

## Recording engineer relation

In [25]:
sql("""
SELECT a.name, COUNT(*) AS cnt
  FROM recording          AS r
  JOIN l_artist_recording AS lar ON r.id = lar.entity1
  JOIN artist             AS a   ON a.id = lar.entity0
  JOIN link               AS l   ON l.id = lar.link
 WHERE l.link_type=128
 GROUP BY a.nameb
 ORDER BY cnt DESC
 LIMIT 20
""")

,name,cnt
0,Rudy van Gelder,5492
1,Antony Howell,1782
2,Tony Faulkner,1274
3,John Cooper,1131
4,Simon Eadon,1121
5,Juha Norlund,941
6,秋田昌美,928
7,Lewis Layton,873
8,Shawn Murphy,778
9,Philip Hobbs,778


## (Recording) producer relation

In [26]:
sql("""
SELECT a.name, COUNT(*) AS cnt
  FROM recording          AS r
  JOIN l_artist_recording AS lar ON r.id = lar.entity1
  JOIN artist             AS a   ON a.id = lar.entity0
  JOIN link               AS l   ON l.id = lar.link
 WHERE l.link_type=141
 GROUP BY a.name
 ORDER BY cnt DESC
 LIMIT 20
""")

,name,cnt
0,Andrew Keener,3207
1,Manfred Eicher,2619
2,Brian Couzens,2394
3,James Mallinson,2186
4,Hans Weber,2184
5,Alfred Lion,2025
6,Walter Legge,1853
7,Martin Sauer,1841
8,James Ginsburg,1830
9,Mark Brown,1679
